In [1]:
from func1 import *
from query1 import *

Using TensorFlow backend.


In [2]:
df = data1(3)
df.index = df['datetime']
df.index = df.index.astype('datetime64[ns]')
df.sort_index(inplace=True)
df.dropna(inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'])

In [3]:
dtat0 = df.iloc[-1:,0:1].values[0][0]
dti = pd.date_range(dtat0, periods=2).shift(1, freq='D')
dfdt = dti.to_frame(index=False, name='datims')

In [4]:
df.drop(('datetime'), axis=1, inplace=True)

In [5]:
from sklearn.preprocessing import MinMaxScaler
y_scaler = MinMaxScaler()
y_scaler.fit(df[['fore24']])
X_scaler = MinMaxScaler()

df[['tail24', 'tail24_avg', 'evap', 'infl', 'losses',
       'rel1', 'rel2', 'rel3', 'rel_tol', 'engr1', 'engr2', 'engr3', 'cond1',
       'cond2', 'cond3', 'str1', 'str2', 'str3', 'run_g1', 'run_g2', 'run_g3',
       'run_c1', 'run_c2', 'run_c3', 'spillway', 'irr', 'camp', 'demand',
       'derate', 'outage', 'stor','fore24']] = X_scaler.fit_transform(df)

In [6]:
dfval = df.values[:,:-1]

In [7]:
n_steps_in, n_steps_out = 3, 2
X_test = dfval.reshape(1,3,31).transpose(0,1,2)

n_features = X_test.shape[2]

In [8]:
model_2 = ShallowConvNet(n_steps_in=n_steps_in, n_steps_out=n_steps_out, n_features=n_features)
model_2.summary()
model_2.load_weights("models/model1_m.hdf5")
model_2.compile(optimizer='adam', loss='mse')
Y_pred = model_2.predict(X_test)
yp = y_scaler.inverse_transform(Y_pred)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2, 64)             4032      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
Total params: 7,384
Trainable params: 7,384
Non-trainable params: 0
_________________________________________________________________


In [11]:
from sqlalchemy import create_engine
user='root'
pwd='opt#565784'
dbname='predictive'
host='10.211.1.25'
port=3306
con = create_engine(f'mysql+pymysql://{user}:{pwd}@{host}:{port}/{dbname}')

In [12]:
df = pd.DataFrame(yp).T
df.columns = ['fore24']

com = pd.concat([dfdt, df], axis=1, join='inner')
com.to_sql('waterpred_m', con, if_exists='replace', index=False)